# Data Cleaning and prepping SCAT values

### Objective

#### Analysis
- Analyze the spreadsheet files
- Understand and explain what each rows and cols are?
- Understand the values and its units
- Consider the columns to keep and justify the columns to be disregarded.

In [34]:
import pandas as pd
pd.set_option('display.max_columns',None)
# CSV raw datasets
SCATS_DATA_OCTOBER_2006_CSV_PATH = 'datasets/Scats Data October 2006.csv'
SCATS_SITE_LISTING_SPREADSHEET_VICROADS_CSV_PATH = 'datasets/SCATSSiteListingSpreadsheet_VicRoads.csv'
TRAFFIC_COUNT_LOCATIONS_WITH_LONG_LAT_CSV_PATH = 'datasets/Traffic_Count_Locations_with_LONG_LAT.csv'

# reading the csv files
df_scats_data_october = pd.read_csv(SCATS_DATA_OCTOBER_2006_CSV_PATH,header = 1)
df_scats_site_listing = pd.read_csv(SCATS_SITE_LISTING_SPREADSHEET_VICROADS_CSV_PATH,header = 9)
df_traffic_long_lat = pd.read_csv(TRAFFIC_COUNT_LOCATIONS_WITH_LONG_LAT_CSV_PATH)

In [30]:
# Outputting SCATS Data October 2006
df_scats_data_october

,SCATS Number,Location,CD_MELWAY,NB_LATITUDE,NB_LONGITUDE,HF VicRoads Internal,VR Internal Stat,VR Internal Loc,NB_TYPE_SURVEY,Date,V00,V01,V02,V03,V04,V05,V06,V07,V08,V09,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,Unnamed: 106,Unnamed: 107,Unnamed: 108
0,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,1/10/06,86,83,52,58,59,44,31,37,30,24,16,24,25,25,15,6,21,17,15,15,16,21,27,21,25,32,61,48,56,66,77,79,67,93,103,130,154,149,210,229,250,246,266,254,300,275,322,292,315,314,308,280,357,298,281,289,345,297,233,227,273,225,265,257,233,244,259,264,265,253,243,210,216,202,177,169,167,136,131,128,118,121,87,113,142,112,114,97,97,66,81,50,59,47,29,34,NaN,NaN,NaN
1,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,2/10/06,32,28,17,11,7,11,6,15,11,12,6,9,1,4,11,9,11,16,22,14,28,26,57,57,70,136,221,196,239,366,355,400,401,400,395,367,315,308,302,306,245,286,279,275,227,234,236,239,320,254,242,261,251,290,254,256,283,226,276,271,281,285,306,301,327,340,294,326,277,382,320,377,259,298,218,190,186,172,161,158,134,141,119,142,103,108,111,102,107,114,80,60,62,48,44,26,NaN,NaN,NaN
2,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,3/10/06,26,32,21,14,10,12,13,10,9,7,8,5,5,6,11,12,8,11,13,10,23,37,64,95,90,183,219,251,302,307,410,351,411,408,405,372,330,366,360,317,312,289,293,299,293,273,264,264,290,260,286,272,274,270,259,313,281,347,280,266,347,277,328,318,357,347,381,288,358,338,365,369,296,335,269,232,211,198,176,163,155,140,135,129,139,146,130,132,114,86,93,90,73,57,29,40,NaN,NaN,NaN
3,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,4/10/06,32,22,28,13,16,8,14,10,8,8,7,6,8,2,7,10,8,14,16,12,24,37,62,84,82,166,230,235,256,336,316,392,374,392,417,380,376,328,324,338,282,272,290,340,301,288,258,272,319,315,251,281,294,301,300,288,282,260,247,252,320,300,264,329,359,345,270,338,308,340,377,335,352,290,258,281,237,203,167,165,133,163,135,141,137,158,115,113,132,101,113,90,78,66,52,44,NaN,NaN,NaN
4,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,5/10/06,40,39,21,11,16,9,15,15,9,6,9,4,4,1,11,9,17,13,16,15,23,35,58,79,81,139,238,235,235,328,344,315,359,434,375,365,356,362,328,330,327,276,311,281,303,324,285,301,331,296,302,299,290,281,308,294,303,296,265,306,312,322,343,317,397,385,388,324,348,361,372,396,315,334,309,271,251,262,228,196,176,152,146,167,122,150,171,120,116,113,99,91,61,55,49,36,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,4821,VICTORIA_ST W OF BURNLEY_ST,002HF02,-37.81296,145.00830,6673,1513,7,1,27/10/06,51,56,43,29,35,19,24,21,15,20,19,14,16,15,13,14,11,15,14,21,17,22,34,51,36,66,109,103,113,136,138,141,218,174,167,168,166,146,163,142,170,158,173,147,194,173,186,151,206,199,218,197,201,197,232,206,205,212,194,215,224,229,253,223,240,261,305,268,303,270,333,264,341,255,214,229,224,204,150,189,164,167,115,133,127,122,122,121,127,103,122,124,117,99,108,88,NaN,NaN,NaN
4188,4821,VICTORIA_ST W OF BURNLEY_ST,002HF02,-37.81296,145.00830,6673,1513,7,1,28/10/06,91,87,89,72,64,81,58,47,52,43,33,47,36,37,27,24,23,35,26,15,16,25,27,24,29,30,38,55,57,50,80,96,79,74,105,110,129,140,154,184,176,163,196,188,193,191,219,224,200,240,223,236,225,200,226,222,216,223,227,219,221,211,236,218,235,242,220,177,201,185,175


- Time-Based Data (e.g., V00, V01, V02, ...): These columns represent traffic counts or flow rates at specific time intervals. They are critical for understanding traffic patterns over time and predicting future traffic flow.
- Date (Start Time): This is important for understanding the day-to-day variation in traffic. It can be useful to include the day of the week or whether the day is a weekday/weekend, as traffic patterns often differ.

Location Information:

- SCATS Number and Location: These provide context on where the data was recorded, allowing the model to learn location-specific traffic patterns.
Coordinates (Latitude and Longitude): If multiple locations are involved, geographical data can help in understanding spatial traffic patterns.

Internal Codes (if meaningful):
- HF VicRoads Internal, VR Internal Stat, VR Internal Loc: If these columns represent different types of traffic flow or control information, they could provide additional context to improve predictions.

In a machine learning model, you'll likely treat the time-based data as your primary features, with additional features such as date, day of the week, and location to provide context. You might also need to consider temporal relationships and how past traffic data influences future traffic predictions.



In [12]:
# Outputting SCATSSiteListingSpreadsheet
df_scats_site_listing

,SCATS Site Listing,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,The list of Site descriptions and correspondin...,NaN,NaN,NaN,NaN
1,If multiple sites are required the format of t...,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
4519,5081,YARRA/LT MALOP,INT,Melway,452C04
4520,4897,YORK/FERRARS,INT,Melway,2K-A01
4521,1296,YOUNG NR PLAYNE,FLASH PX,Melway,100AC08
4522,741,YOUNG NR STATION,POS,Melway,102D02


Columns:
- Site Number: This likely represents the unique identifier for each SCATS (Sydney Coordinated Adaptive Traffic System) site.
- Location Description: Describes the physical location of the SCATS site, such as the intersection or road segment it monitors.
- Site Type: Indicates the type of site, which might be "Intersection" (INT) or other designations.
- Directory: Possibly refers to a reference directory or map that includes the SCATS site.
- Map Reference: A specific reference on a map (e.g., Melway) to locate the site geographically.

Rows:

   Each row after the headers appears to correspond to a SCATS site with details about its location, type, and reference information.

   This dataset provides geographical and descriptive information about SCATS sites, which could be useful in your traffic prediction model by linking traffic data to specific sites and their characteristics.

In [14]:
# Outputting Traffic_Count_Locations_with_LONG_LAT 
df_traffic_long_lat

,X,Y,FID,OBJECTID,TFM_ID,TFM_DESC,TFM_TYP_DE,MOVEMENT_T,SITE_DESC,ROAD_NBR,DECLARED_R,LOCAL_ROAD,DATA_SRC_C,DATA_SOURC,TIME_CATEG,YEAR_SINCE,LAST_YEAR,AADT_ALLVE,AADT_TRUCK,PER_TRUCKS
0,144.250614,-36.779313,7001,7301,7656,CALDER HWY NE OF OAK ST,INTERSECTION,All Moves,CALDER HWY & OAK ST,2530,CALDER HIGHWAY,HIGH STREET,TMVMT,Manual,Greater than 10 Years,19,1997,7700,330,0.04
1,145.356779,-37.835309,7002,7302,29406,MT DANDENONG RD S BD SE OF UPALONG RD,INTERSECTION,All Moves,MT DANDENONG RD SE OF UPALONG RD,4991,MOUNT DANDENONG ROAD,MOUNT DANDENONG TOURIST ROAD,APARX,Classification,Greater than 10 Years,16,2000,1900,0,0.00
2,144.988844,-37.824629,7003,7303,22676,SWAN ST W BD E OF PUNT RD,INTERSECTION,All Moves,PUNT RD LEFT TURN TO SWAN ST OD:12,2080,HODDLE HIGHWAY,PUNT ROAD,MOTSV,Manual,Greater than 10 Years,23,1993,15000,630,0.04
3,144.932442,-37.803783,7004,7304,27902,DYNON RD W BD E OF RADCLIFFE ST,INTERSECTION,All Moves,DYNON RD & RADCLIFFE ST,5035,DYNON ROAD,DYNON ROAD,TMVMT,Manual,Greater than 10 Years,19,1997,12000,1300,0.10
4,145.030601,-37.660502,7005,7305,10935,DALTON RD N of CHILDS RD,INTERSECTION,All Moves,CHILDS RD & DALTON RD,5605,DALTON ROAD,DALTON ROAD,TMVMT,Manual,Greater than 10 Years,19,1997,12000,600,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57942,144.306938,-36.728799,56996,56399,60461,DLAND HWY N BD BTW BOSQUET ST AND PLUMRIDGE ST,MIDBLOCK,All Moves,DLAND HWY BTW BOSQUET ST AND PLUMRIDGE ST,2590,MIDLAND HIGHWAY,NAPIER STREET,APARX,Classification,Greater than 10 Years,0,0,0,0,0.00
57943,146.764140,-38.021751,56997,56899,59432,TRARALGON-MAFFRA ROAD(F) 52 m 7 from ROSEDALE-...,DEPARTURE,All Moves,SEDALE-HEYFIELD ROAD AND TRARALGON-MAFFRA ROAD,5543,TRARALGON-MAFFRA ROAD,TRARALGON-MAFFRA ROAD,TMVMT,Manual,Greater than 10 Years,0,0,0,0,0.00
57944,145.062372,-37.846644,56998,56700,50953,TOORAK_RD E of BOWEN_ST,INTERSECTION,All Moves,TOORAK RD NR BOWEN ST,2750,BURWOOD HIGHWAY,TOORAK ROAD,SCATS,Scats,Greater than 10 Years,26,1990,18000,0,0.00
57945,146.765214,-38.022690,56999,56900,59433,ROSEDALE-HEYFIELD ROAD(F) 91 m 4 from THREE CH...,APPROACH,All Moves,SEDALE-HEYFIELD ROAD AND TRARALGON-MAFFRA ROAD,5544,ROSEDALE-HEYFIELD ROAD,ROSEDALE-HEYFIELD ROAD,TMVMT,Manual,Greater than 10 Years,0,0,0,0,0.00


The file "Traffic_Count_Locations_with_LONG_LAT.csv" contains the following columns:

- X and Y: These are the geographic coordinates (longitude and latitude, respectively) of the traffic count locations.
- FID and OBJECTID: These are identifiers that may refer to specific features or objects within a geographic or mapping system.
- TFM_ID: This is an identifier for the traffic flow monitoring site.
- TFM_DESC: Description of the traffic monitoring site, typically indicating the road or intersection being monitored.
- TFM_TYP_DE: Type of the traffic flow monitoring, which seems to describe the location type (e.g., "INTERSECTION").
- MOVEMENT_T: Describes the movement type being monitored (e.g., "All Moves" likely means all traffic directions).
- SITE_DESC: A more detailed description of the site, often including road names or specific intersections.
- ROAD_NBR: The road number for the monitored site.
- DECLARED_R: The declared road name or type, often indicating the official name or designation of the road.
- LOCAL_ROAD: Describes the local road connected to the declared road, giving further context to the location.
- DATA_SRC_C: Source of the data, likely indicating the method of data collection.
- DATA_SOURC: Further description of the data source, such as whether it was manually collected or obtained through a specific method.
- TIME_CATEG: Indicates the time category, such as how old the data is (e.g., "Greater than 10 Years").
- YEAR_SINCE: The number of years since the data was first collected.
- LAST_YEAR: The last year in which the data was updated.
- AADT_ALLVE: The Annual Average Daily Traffic (AADT) for all vehicles at the site.
- AADT_TRUCK: The AADT specifically for trucks at the site.
- PER_TRUCKS: The percentage of total traffic that consists of trucks.

Rows:

Each row in this file corresponds to a specific traffic count location. The data in each row provides detailed information about that location, including geographic coordinates, traffic flow details, and historical traffic data.

This file provides important context that can be combined with time-based traffic data to enhance the accuracy of your traffic flow prediction model. For instance, the geographic coordinates and AADT values can help in understanding spatial and volume-based traffic patterns.

__________________________________________________________________________________________


# Data Cleaning & Feature Engineering

#### Dropping empty columns and fill missing values with 0

In [32]:
df_scats_data_october = df_scats_data_october.dropna(how='all', axis=1)
# dropped wrong valued entries
df_scats_data_october = df_scats_data_october[df_scats_data_october['HF VicRoads Internal'] != -1]

#### Converting values in table to integers
- By default, values in csv files are strings. We convert the data types to int to be safe

In [ ]:
numerical_columns = []
for i in range(96):
    if i < 10:
        numerical_columns.append('V0' + str(i))
    else:
        numerical_columns.append('V' + str(i))

df_scats_data_october[numerical_columns] = df_scats_data_october[numerical_columns].apply(pd.to_numeric,
                                                                                          errors='coerce')


#### Adding Features and Column Formatting
- reformat the date column
- add a 'Day_Of_Week' column as a feature (will be useful for traffic flow comparison among the different days)
- add a 'Is_Weekend' column as a feature (will be useful for traffic flow comparison between it and the weekdays) 

In [ ]:
# Correctly parse the Date column with the adjusted format
df_scats_data_october['Date'] = pd.to_datetime(df_scats_data_october['Date'], format='%d/%m/%y', dayfirst=True)

# Create the new columns
new_date_columns = pd.DataFrame({
    'Day_Of_Week': df_scats_data_october['Date'].dt.dayofweek,
    'Is_Weekend': df_scats_data_october['Date'].dt.dayofweek >= 5
})

# Concatenate the new columns with the original DataFrame
df_scats_data_october = pd.concat([df_scats_data_october, new_date_columns], axis=1)


#### Rolling Daily Quarter Average

- A rolling average is often used to smooth out short-term fluctuations and highlight longer-term trends or patterns. The purpose is to smooths our short-term fluctuations that might occur in the intervals. This can help identify the overall trend or pattern in traffic flow more clearly
- In our case, each quarter average equates to 6 hours of traffic data. (might need to expand this into 1 hour intervals)

In [ ]:
df_scats_data_october['Rolling_First_Quarter'] = df_scats_data_october[numerical_columns].iloc[:, :24].mean(axis=1)
df_scats_data_october['Rolling_Second_Quarter'] = df_scats_data_october[numerical_columns].iloc[:, 24:48].mean(axis=1)
df_scats_data_october['Rolling_Third_Quarter'] = df_scats_data_october[numerical_columns].iloc[:, 48:72].mean(axis=1)
df_scats_data_october['Rolling_Fourth_Quarter'] = df_scats_data_october[numerical_columns].iloc[:, 72:96].mean(axis=1)


In [ ]:
df_scats_data_october